# Differential gene targeting of PDAC tumor subtypes
Author: Deborah Weighill<sup>1</sup>, Marouen Ben Guebila<sup>1</sup>

<sup>1</sup> Department of Biostatistics, Harvard T.H. Chan School of Public Health, Boston, MA.

# Introduction

Pancreatic ductal adenocarcinoma (PDAC)  is the fourth leading leading cause of cancer worldwide <sup>1</sup>, with few efficient therapeutic options. In fact, PDAC is a highly hetergenous disease, this is why one size fits all approach to treatment have provided limited success so far. Therefore, subtype identification is necessary to tailor therapeutic approaches to the specific molecular characteristic of disease progression in each patient.

Recent work<sup>2,3</sup> has suggested novel subtypes in PDAC by the virtual dissection of the genomic markers. It has been suggested that gene expression could be an informative feature in classifying patients<sup>4</sup>. In addition, gene regulation could play a major role in the emergence of distinct phenotypes in PDAC. Therefore, modeling regulatory features in PDAC could provide a novel approach to stratify patients and provide tailored therapeutic apporaches.

![https://www.pancreaticcancer.org.uk/information/just-diagnosed-with-pancreatic-cancer/types-of-pancreatic-cancer/](https://www.pancreaticcancer.org.uk/wp-content/uploads/2020/09/anatomy-of-pancreas-final-sept-2016-1-1024x676.jpg)

In this analysis, we will study PDAC hetergenity by building gene regulatory networks using gene expression data from TCGA<sup>5</sup>. We will also use data provided by a novel classification approach<sup>3</sup> to startify patients by the classical and basal-like subtype. Finally, we will compute a network metric called the gene targeting score to identify differentially regulated genes between basal-like and classical subtypes.

# Loading data and libraries

## Loading the packages

First, we need to install the [netZooR package](netzoo.github.io) that has the network analysis tools.

In [ ]:
# install.packages("devtools") 
library(devtools)
# install netZooR pkg with vignettes, otherwise remove the "build_vignettes = TRUE" argument.
#devtools::install_github("netZoo/netZooR", build_vignettes = TRUE)

Then, we can load the packages that are needed for the analysis.

In [ ]:
library(netZooR) # to use network reconstruction methods PANDA and LIONESS
library(recount)
library(limma) # to compute differential targeting 
library(tidyr)
library(data.table)
library(reticulate) # to call Python functions
#use_python("/home/ubuntu/.local/bin/venv/bin/python3") # Please replace the path in the reticulate command use_python with you Python installation path.

## Load gene expression data
Now, let's load TCGA PDAC gene expression data, which was obtained from [recount2](https://jhubiostatistics.shinyapps.io/recount/)<sup>6</sup>. The recount project has processed all the publicly availble gene expression data using a uniform pipeline, which allows to control the variance between the studies and extract biological signal with minimal interference of effects due to the analysis method.

First, we need to to normalize the counts to consider the coverage of all the samples using the Transcript Per Million method (TPM). This method takes into account sequencing depth and gene length to normalize gene expression<sup>7</sup>.

In [ ]:
# Load gene expression object downloaded from recount.
load("/opt/data/tttutorial/rse_gene_pancreas.Rdata")

#convert to TPMs
TPM <- getTPM(rse_gene, length_var = "bp_length", mapped_var = 'mapped_read_count')
rownames(TPM)<-sub("\\.[0-9]*","", rownames(TPM))

Then, we need to map TCGA ID to samples and further clean the metadata.

In [ ]:
#extract meta data and map RNA-seq file name to TCGA ID
metadata <- rse_gene@colData
metadata <- metadata[,c("gdc_cases.samples.portions.analytes.aliquots.submitter_id","gdc_cases.demographic.gender","gdc_cases.demographic.race","gdc_cases.demographic.ethnicity","gdc_cases.exposures.weight","gdc_cases.exposures.height","gdc_cases.project.primary_site","gdc_cases.diagnoses.primary_diagnosis","cgc_case_age_at_diagnosis","gdc_cases.diagnoses.tumor_stage","gdc_cases.diagnoses.days_to_death","gdc_cases.diagnoses.vital_status")]
metadata$rnaseq_column <- rownames(metadata)
metadata <- separate(data.frame(metadata), gdc_cases.samples.portions.analytes.aliquots.submitter_id, c("proj", "tss", "participant", "sample","portion","plate","center"), "-",remove = FALSE)
metadata$sampletype <- as.numeric(substr(metadata$sample,1,2))
colnames(TPM) <- metadata$gdc_cases.samples.portions.analytes.aliquots.submitter_id[match(colnames(TPM),metadata$rnaseq_column)]

# Extract samples with subtypes

TCGA includes samples taken from the site of tumor but also samples from the healthy tissues around the tumor. These samples are called adjacent normal samples and we need to process the data to remove them from our analysis.

Using TCGA barcodes (see https://docs.gdc.cancer.gov/Encyclopedia/pages/TCGA_Barcode/), we extract only samples corresponding to tumor (i.e. removing adjacent normal tissue).

In [ ]:
metadata$tumor <- ifelse(((metadata$sampletype >= 1) & (metadata$sampletype <= 9)),"tumor","non_tumor")
tumor_samples <- metadata$gdc_cases.samples.portions.analytes.aliquots.submitter_id[which(metadata$tumor == "tumor")]
selected_samples_pdac <- TPM[,tumor_samples]

Now we need to convert gene expression data into a data frame to be able to work with it in the network modeling step. Also, we need to annotate the columns and rows of the expression matrix.

In [ ]:
# load the data and parse the TCGA ids
samples <- as.data.frame(tumor_samples)
colnames(samples) <- "sampleID"
samples <- separate(samples, sampleID, c("proj", "tss", "participant","sample","portion","plate","center"), "-", remove = FALSE)
samples$shorterID <- paste(samples[,2],samples[,3],samples[,4],samples[,5], sep = "-")
colnames(selected_samples_pdac) <- samples$shorterID

We now load the previously annotation file of the molecular subtypes of the samples, as determined in Moffit et al.<sup>3</sup>. We narrow down our samples to those assigned a subtype. In this example, we focus on genes which vary in their expression the most, and thus chose those with $std(logTPM) \geq 0.4$. If one has access to enough memory and computational time, one can use the whole genome.

First, we start by loading and exploring the subtype assignment file.

In [ ]:
sample_subtypes <- read.table("/opt/data/tttutorial/subtypes.txt", header = FALSE)
colnames(sample_subtypes) <- c("sample","subtype")
head(sample_subtypes)

Then, we filter the samples based on the criteria we specified earlier.

In [ ]:
subtype_samples <- as.vector(sample_subtypes$sample[which(!(is.na(sample_subtypes$subtype)))])
tpms_selected_samples <- selected_samples_pdac[,subtype_samples]
rowsums <- apply(data.matrix(tpms_selected_samples),MARGIN = 1, FUN = sum)
tpms_selected_samples_genes <- tpms_selected_samples[names(which(rowsums != 0)),]
logtpms_selected_samples_genes <- log10(data.matrix(tpms_selected_samples_genes))
sdev <- apply(data.matrix(logtpms_selected_samples_genes),MARGIN = 1, FUN = sd)
logtpms_selected_samples_genes_sd <- logtpms_selected_samples_genes[names(which(sdev >= 0.4)),]
write.table(logtpms_selected_samples_genes_sd,file = "pdac_expression_sd_log_04.txt", row.names = TRUE, col.names = FALSE, quote=FALSE,sep = "\t")

# Network analysis

## Reconstruction of single-sample gene regulatory networks
Now, we build an aggregate gene regulatory network for all the samples in the dataset using PANDA<sup>8</sup>. Then we dissect infer a single-sample network from the single-sample expression and the aggregate network using LIONESS<sup>9</sup>.

These methods take as an input inputs three data types: the gene expression data from TCGA, a naive motif prior (generated from FIMO<sup>10</sup> scans of the human genome), and a protein-protein interaction network from STRINGdb<sup>11</sup>. The PPI and motif prior network were processed as in [Sonawane et al.](https://doi.org/10.1016/j.celrep.2017.10.001)<sup>12</sup>.

We will call the `lioness.py` function here, which will call PANDA underneath the hood. This is a Python implementation that is called from R using the `reticulate` package, so make sure that Python path is correctly speicifed in the `use_python` command. Otherwise, please check the README file of [netZooR](https://github.com/netZoo/netZooR) for detailed instructions.

In [ ]:
lioness_pdac <- lioness.py(expr_file = "./pdac_expression_sd_log_04.txt" , motif_file = "/opt/data/tttutorial/motif_prior.txt", ppi_file = "/opt/data/tttutorial/ppi.txt")
#save(lioness_pdac, file = "lioness_pdac.RData")

## Calculate gene targeting score
Now, for each gene in each sample-specific network, we calculate the gene targeting score as the degree (sum of adjacent edge weights) in that sample's network. 

In [ ]:
map <- read.table("/opt/data/tttutorial/geneID_name_map.txt")
sample_order <- colnames(logtpms_selected_samples_genes_sd)
subtype_order <- sample_subtypes$subtype[match(sample_order,sample_subtypes$sample)]
subtype_order_names <- ifelse(subtype_order == 1,"basal","classical")
colnames(lioness_pdac) <- c("TF","Gene",sample_order)
geneTargetting <- aggregate(. ~ Gene, lioness_pdac[-1], sum)

## Differential targeting
Finally, We use limma<sup>13</sup> ([Ritchie et al.](https://doi.org/10.1093/nar/gkv007)) to identify differentially targeted genes between basal-like and classical samples, i.e., the genes that have a differential targeting score in each of the subtype.

In [ ]:
gene_targetting_matrix <- geneTargetting[,c(2:151)]
rownames(gene_targetting_matrix) <- rownames(geneTargetting)
subtype <- as.factor(subtype_order_names)
design <- model.matrix(~subtype)
fit = lmFit(as.matrix(gene_targetting_matrix),design)
fit = eBayes(fit)

Then, we can build our result table and see which genes are differentially targeted across samples.

In [ ]:
adjustedTarg <- topTable(fit, number = Inf, adjust = "fdr", coef = "subtypeclassical")
adjustedTarg$Gene <- geneTargetting$Gene[as.numeric(rownames(adjustedTarg))]
head(adjustedTarg)

# Conclusion
A complete biological interepretation of the resultls and the rationale behind the use of targeting in finding difefremtial regulatory patterns is described in [our publication](https://arxiv.org/abs/2101.03985) that comes with this tutorial. In particular, we showed that targeting is a network score that can reveal new biological insights not covered by coexpression networks, and differential gene expression analysis.

# References

1- Adamska, Aleksandra, Alice Domenichini, and Marco Falasca. "Pancreatic ductal adenocarcinoma: current and evolving therapies." International journal of molecular sciences 18.7 (2017): 1338.

2- Rashid, Naim U., et al. "Purity independent subtyping of tumors (PurIST), A clinically robust, single-sample classifier for tumor subtyping in pancreatic cancer." Clinical Cancer Research 26.1 (2020): 82-92.

3- Moffitt, Richard A., et al. "Virtual microdissection identifies distinct tumor-and stroma-specific subtypes of pancreatic ductal adenocarcinoma." Nature genetics 47.10 (2015): 1168.

4- Diaferia, Giuseppe R., et al. "Dissection of transcriptional and cis‐regulatory control of differentiation in human pancreatic cancer." The EMBO journal 35.6 (2016): 595-617.

5- Weinstein, John N., et al. "The cancer genome atlas pan-cancer analysis project." Nature genetics 45.10 (2013): 1113.

6- Collado-Torres, Leonardo, et al. "Reproducible RNA-seq analysis using recount2." Nature biotechnology 35.4 (2017): 319-321.

7- Li, Bo, et al. "RNA-Seq gene expression estimation with read mapping uncertainty." Bioinformatics 26.4 (2010): 493-500.

8- Glass, Kimberly, et al. "Passing messages between biological networks to refine predicted interactions." PloS one 8.5 (2013): e64832.

9- Kuijjer, Marieke Lydia, et al. "Estimating sample-specific regulatory networks." Iscience 14 (2019): 226-240.

10- Grant, Charles E., Timothy L. Bailey, and William Stafford Noble. "FIMO: scanning for occurrences of a given motif." Bioinformatics 27.7 (2011): 1017-1018.

11- Szklarczyk, Damian, et al. "STRING v11: protein–protein association networks with increased coverage, supporting functional discovery in genome-wide experimental datasets." Nucleic acids research 47.D1 (2019): D607-D613.

12- Sonawane, Abhijeet Rajendra, et al. "Understanding tissue-specific gene regulation." Cell reports 21.4 (2017): 1077-1088.

13- Ritchie, Matthew E., et al. "limma powers differential expression analyses for RNA-sequencing and microarray studies." Nucleic acids research 43.7 (2015): e47-e47.

14- Weighill, Deborah, et al. "Gene targeting in disease networks" arXiv 2101.03985 (2021). https://arxiv.org/abs/2101.03985


## Image credit 

https://www.pancreaticcancer.org.uk/information/just-diagnosed-with-pancreatic-cancer/types-of-pancreatic-cancer/